# Instructions
You will be peforming an ETL process:
* Extract the data from S3
* Perform transformations to clean up the data 
* Perform analysis to create 4 datasets required by the business 
* Load the results into Snowflake

### Extract the data from source 
* Read the `accidents_2017_to_2023_english_csv` from the S3 bucket in `techcatalyst-raw/accidents`

### Renaming
__Rename Columns__: For better readability and usability, rename the columns to be more descriptive and follow a consistent naming convention. 
* `inverse_data` --> `accident_date`
* `week_day` --> `day_of_week` 
* `hour` --> `accident_time` 
* `state` --> `state_code` 
* `road_id` --> `highway_number` 
* `city` --> `city_name` 
* `cause_of_accident` --> `primary_cause` 
* `type_of_accident` --> `accident_type` 
* `victims_condition` --> `casualty_status` 
* `weather_timestamp` --> `daylight_condition` 
* `road_direction` --> `traffic_direction` 
* `wheather_condition` --> `weather_condition`   
* `road_type` --> `highway_type` 
* `road_delineation` --> `road_geometry` 
* `people` --> `total_people_involved` 
* `deaths` --> `fatalities` 
* `slightly_injured` --> `minor_injuries` 
* `severely_injured` --> `serious_injuries` 
* `uninjured` --> `uninjured_count` 
* `ignored` --> `unknown_status` 
* `total_injured` --> `total_casualties` 
* `vehicles_involved` --> `vehicle_count` 
* `latitude` --> `accident_latitude` 
* `longitude` --> `accident_longitude` 
* `regional` --> `regional_office` 
* `police_station` --> `reporting_station`

### Fix data types and inconsistencies
* __Date/Time__: Convert `accident_date` to a proper datetime format. Standardize `hour` to datetime or extract hour as an integer for easier analysis.
* __Numeric Conversion__: Convert the km column to a numeric float type. This requires handling the comma decimal separator by replacing it with a period (.) before conversion.
    * Convert `km` to numeric (handle mixed formats like "43,4" vs 320)
* __Text Fields__: Standardize all object/string columns (e.g., `accident_cause`, `city`, `road_direction`) by converting them to lowercase and stripping leading/trailing whitespace. This prevents issues with duplicate categories (e.g., "sunny" vs. "Sunny").

### Data Quaulity (DQ) Checks 
Before analysis, it's vital to assess the data's integrity.

### Handle missing values
* Document which columns contain missing data. Count missing values in each column and document which ones have gaps 
* For simplicity in this project, remove rows where these key columns have missing data using
### Outliers
*  Examine numerical fields and look for values that seem physically impossible or are extreme statistical outliers (e.g., an accident involving 100 vehicles). `Boxplots` are a great way to visualize these. Document interesting findings.
* Document these columns 
### Data Validation (DQ)
* Create a new boolean column `total_injured_match_ind` to verify if the sum of `slightly_injured` and `severely_injured` equals the `total_injured` column

### Feature Engieering
* __Date Components__: Extract year, month, day, and quarter from the `accident_date`.
* __Weekend vs. Weekday__: Create a `weekend_flag` (True/False) based on the day_of_week.
* Time of Day Category: Create a time_of_day_category column based on the hour:
    * Morning (5am-12pm)
    * Afternoon (12pm-5pm)
    * Evening (5pm-9pm)
    * Night (9pm-5am)
* __Fatality & Vehicle Flags__:
    * `fatal_accident_flag`: True if deaths > 0
    * `multi_vehicle_flag`: True if vehicles_involved > 1
* __Accident Severity Score__: Create the accident_severity_score using the specified formula to quantify the overall severity of an accident:
    * `accident_severity_score` = (deaths * 10) + (severely_injured * 3) + slightly_injured

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from datetime import datetime
import numpy as np
from dotenv import load_dotenv

load_dotenv()



False

In [ ]:
def load_accident_data():
    # YOUR CODE HERE: Load the CSV file into a DataFrame
    print(f"Loaded {len(df)} records with {len(df.columns)} columns")
    
    return df

df = load_accident_data()

Loaded 463152 records with 27 columns


In [91]:
df.head(5)

,inverse_data,week_day,hour,state,road_id,km,city,cause_of_accident,type_of_accident,victims_condition,weather_timestamp,road_direction,wheather_condition,road_type,road_delineation,people,deaths,slightly_injured,severely_injured,uninjured,ignored,total_injured,vehicles_involved,latitude,longitude,regional,police_station
0,2017-01-01,sunday,01:45:00,RS,116.0,"34,9",VACARIA,Mechanical loss/defect of vehicle,Rear-end collision,With injured victims,Night,Decreasing,Clear sky,Simple,Straight,6,0,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS
1,2017-01-01,sunday,01:00:00,PR,376.0,636,TIJUCAS DO SUL,Incompatible velocity,Run-off-road,With dead victims,Night,Increasing,Drizzle,Double,Curve,2,1,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR
2,2017-01-01,sunday,04:40:00,BA,101.0,65,ENTRE RIOS,Driver was sleeping,Head-on collision,With dead victims,Sunrise,Decreasing,Cloudy,Simple,Curve,5,1,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA
3,2017-01-01,sunday,06:30:00,PA,316.0,"72,5",CASTANHAL,Driver's lack of attention to conveyance,Side impact collision,With dead victims,Sunrise,Decreasing,Clear sky,Simple,Straight,4,1,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA
4,2017-01-01,sunday,09:00:00,GO,20.0,"220,5",POSSE,Road's defect,Collision with fixed object,With injured victims,Day,Decreasing,Clear sky,Simple,Temporary Detour,3,0,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF


In [92]:
df.columns

Index(['inverse_data', 'week_day', 'hour', 'state', 'road_id', 'km', 'city',
       'cause_of_accident', 'type_of_accident', 'victims_condition',
       'weather_timestamp', 'road_direction', 'wheather_condition',
       'road_type', 'road_delineation', 'people', 'deaths', 'slightly_injured',
       'severely_injured', 'uninjured', 'ignored', 'total_injured',
       'vehicles_involved', 'latitude', 'longitude', 'regional',
       'police_station'],
      dtype='object')

In [ ]:
def rename_columns(df):
    """Rename columns for better readability and consistency"""
    column_mapping = {
        # YOUR CODE HERE: Define the mapping of old column names to new column names
    }
    
    df_renamed = df.rename(columns=column_mapping)
    print("✓ Columns renamed successfully")
    return df_renamed

In [94]:
df_renamed = rename_columns(df)
df_renamed.head(5)

✓ Columns renamed successfully


,accident_date,day_of_week,accident_time,state_code,highway_number,kilometer_marker,city_name,accident_cause,accident_type,casualty_status,daylight_condition,traffic_direction,weather_condition,road_classification,road_geometry,total_people_involved,fatalities,minor_injuries,serious_injuries,uninjured_count,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station
0,2017-01-01,sunday,01:45:00,RS,116.0,"34,9",VACARIA,Mechanical loss/defect of vehicle,Rear-end collision,With injured victims,Night,Decreasing,Clear sky,Simple,Straight,6,0,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS
1,2017-01-01,sunday,01:00:00,PR,376.0,636,TIJUCAS DO SUL,Incompatible velocity,Run-off-road,With dead victims,Night,Increasing,Drizzle,Double,Curve,2,1,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR
2,2017-01-01,sunday,04:40:00,BA,101.0,65,ENTRE RIOS,Driver was sleeping,Head-on collision,With dead victims,Sunrise,Decreasing,Cloudy,Simple,Curve,5,1,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA
3,2017-01-01,sunday,06:30:00,PA,316.0,"72,5",CASTANHAL,Driver's lack of attention to conveyance,Side impact collision,With dead victims,Sunrise,Decreasing,Clear sky,Simple,Straight,4,1,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA
4,2017-01-01,sunday,09:00:00,GO,20.0,"220,5",POSSE,Road's defect,Collision with fixed object,With injured victims,Day,Decreasing,Clear sky,Simple,Temporary Detour,3,0,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF


In [95]:
df_renamed.columns

Index(['accident_date', 'day_of_week', 'accident_time', 'state_code',
       'highway_number', 'kilometer_marker', 'city_name', 'accident_cause',
       'accident_type', 'casualty_status', 'daylight_condition',
       'traffic_direction', 'weather_condition', 'road_classification',
       'road_geometry', 'total_people_involved', 'fatalities',
       'minor_injuries', 'serious_injuries', 'uninjured_count',
       'unknown_status', 'total_casualties', 'vehicle_count',
       'accident_latitude', 'accident_longitude', 'regional_office',
       'reporting_station'],
      dtype='object')

In [96]:
df_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463152 entries, 0 to 463151
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   accident_date          463152 non-null  object 
 1   day_of_week            463152 non-null  object 
 2   accident_time          463152 non-null  object 
 3   state_code             463152 non-null  object 
 4   highway_number         462162 non-null  float64
 5   kilometer_marker       462162 non-null  object 
 6   city_name              463152 non-null  object 
 7   accident_cause         463152 non-null  object 
 8   accident_type          463152 non-null  object 
 9   casualty_status        463152 non-null  object 
 10  daylight_condition     463152 non-null  object 
 11  traffic_direction      463152 non-null  object 
 12  weather_condition      463152 non-null  object 
 13  road_classification    463152 non-null  object 
 14  road_geometry          463152 non-nu

In [97]:
df_renamed.dtypes

accident_date             object
day_of_week               object
accident_time             object
state_code                object
highway_number           float64
kilometer_marker          object
city_name                 object
accident_cause            object
accident_type             object
casualty_status           object
daylight_condition        object
traffic_direction         object
weather_condition         object
road_classification       object
road_geometry             object
total_people_involved      int64
fatalities                 int64
minor_injuries             int64
serious_injuries           int64
uninjured_count            int64
unknown_status             int64
total_casualties           int64
vehicle_count              int64
accident_latitude        float64
accident_longitude       float64
regional_office           object
reporting_station         object
dtype: object

In [ ]:
def fix_data_types(df):
    """Fix data types and handle inconsistencies"""
    
    # Convert accident_date to datetime
    # YOUR CODE HERE: Convert 'accident_date' to datetime format
    
    # Extract hour from accident_time or convert to integer hour
    # YOUR CODE HERE: Convert 'accident_time' to integer hour and store as 'accident_hour'
    
    # Convert kilometer_marker to numeric (handle "43,4" format)
    # YOUR CODE HERE: Convert 'kilometer_marker' to numeric, replacing commas with dots
    # hint: consider format='%H:%M:%S', errors='coerce' when using pd.to_datetime
    
    # Standardize text fields (lowercase, strip whitespace)
    # YOUR CODE HERE: Define text columns to standardize
    
    # Example of text columns to standardize
    for col in text_columns:
        if col in df.columns:
            df[col] =   # YOUR CODE HERE: Standardize text columns to lowercase and strip whitespace
    
    print("✓ Data types fixed and standardized")
    return df

In [99]:
df_renamed_fixed = fix_data_types(df_renamed)
df_renamed_fixed.dtypes

✓ Data types fixed and standardized


accident_date            datetime64[ns]
day_of_week                      object
accident_time                    object
state_code                       object
highway_number                  float64
kilometer_marker                float64
city_name                        object
accident_cause                   object
accident_type                    object
casualty_status                  object
daylight_condition               object
traffic_direction                object
weather_condition                object
road_classification              object
road_geometry                    object
total_people_involved             int64
fatalities                        int64
minor_injuries                    int64
serious_injuries                  int64
uninjured_count                   int64
unknown_status                    int64
total_casualties                  int64
vehicle_count                     int64
accident_latitude               float64
accident_longitude              float64


In [100]:
df_renamed_fixed.head()

,accident_date,day_of_week,accident_time,state_code,highway_number,kilometer_marker,city_name,accident_cause,accident_type,casualty_status,daylight_condition,traffic_direction,weather_condition,road_classification,road_geometry,total_people_involved,fatalities,minor_injuries,serious_injuries,uninjured_count,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station,accident_hour
0,2017-01-01,sunday,01:45:00,RS,116.0,34.9,vacaria,Mechanical loss/defect of vehicle,rear-end collision,with injured victims,Night,Decreasing,clear sky,Simple,straight,6,0,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS,1
1,2017-01-01,sunday,01:00:00,PR,376.0,636.0,tijucas do sul,Incompatible velocity,run-off-road,with dead victims,Night,Increasing,drizzle,Double,curve,2,1,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR,1
2,2017-01-01,sunday,04:40:00,BA,101.0,65.0,entre rios,Driver was sleeping,head-on collision,with dead victims,Sunrise,Decreasing,cloudy,Simple,curve,5,1,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA,4
3,2017-01-01,sunday,06:30:00,PA,316.0,72.5,castanhal,Driver's lack of attention to conveyance,side impact collision,with dead victims,Sunrise,Decreasing,clear sky,Simple,straight,4,1,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA,6
4,2017-01-01,sunday,09:00:00,GO,20.0,220.5,posse,Road's defect,collision with fixed object,with injured victims,Day,Decreasing,clear sky,Simple,temporary detour,3,0,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF,9


In [ ]:
def perform_data_quality_checks(df):
    """Comprehensive data quality assessment"""
    
    print("=== DATA QUALITY REPORT ===")
    
    # 1. Missing Values Documentation
    # YOUR CODE HERE: Count missing values in each column

    # if there are missing data, loop through each column and print the number of missing values and percentage
    # percentage is calculated as (missing_count / total_rows) * 100
    
    # 2. Remove rows with missing data
    # YOUR CODE HERE: Remove rows with any missing values
    print(f"\n   Removed {removed_rows} rows with missing data")
    
    # 3. Outlier Detection
    print("\n2. OUTLIER ANALYSIS:")
    numerical_cols = ['total_people_involved', 'fatalities', 'minor_injuries', 
                     'serious_injuries', 'total_casualties', 'vehicle_count']
    
    outlier_summary = {}
    # Outlier detection is done for you using the IQR method
    # THIS IS A SIMLPE METHOD AND MAY NOT BE SUITABLE FOR ALL DATASETS 
    # YOU WILL LEARN ABOUT IQR IN A LATER LESSON
    for col in numerical_cols:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)]
            outlier_summary[col] = {
                'count': len(outliers),
                'percentage': (len(outliers) / len(df_clean)) * 100,
                'max_value': df_clean[col].max(),
                'realistic_max': upper_bound
            }
    
    for col, stats in outlier_summary.items():
        print(f"   {col}: {stats['count']} outliers ({stats['percentage']:.2f}%)")
        print(f"      Max value: {stats['max_value']}, Expected max: {stats['realistic_max']:.1f}")
    
    # 4. Data Quality Check: total_injured calculation
    print("\n3. DATA CONSISTENCY CHECK:")
    df_clean['calculated_total'] = # YOUR CODE HERE: Calculate total casualties as sum of fatalities, minor injuries, and serious injuries
    df_clean['total_match_ind'] = # YOUR CODE HERE: Check if calculated total matches total_casualties
    
    mismatches = (~df_clean['total_match_ind']).sum()
    print(f"   Total casualties mismatch: {mismatches} records ({(mismatches/len(df_clean)*100):.2f}%)")
    
    return df_clean

In [102]:
df_clean = perform_data_quality_checks(df_renamed_fixed)
df_clean.head()

=== DATA QUALITY REPORT ===

1. MISSING VALUES:
Columns with missing values:
   reporting_station: 1310 missing (0.28%)
   highway_number: 990 missing (0.21%)
   kilometer_marker: 990 missing (0.21%)
   regional_office: 10 missing (0.00%)

   Removed 2223 rows with missing data

2. OUTLIER ANALYSIS:
   total_people_involved: 8545 outliers (1.85%)
      Max value: 80, Expected max: 6.0
   fatalities: 31212 outliers (6.77%)
      Max value: 21, Expected max: 0.0
   minor_injuries: 20205 outliers (4.38%)
      Max value: 61, Expected max: 2.5
   serious_injuries: 96315 outliers (20.90%)
      Max value: 31, Expected max: 0.0
   total_casualties: 30354 outliers (6.59%)
      Max value: 66, Expected max: 2.5
   vehicle_count: 8558 outliers (1.86%)
      Max value: 23, Expected max: 3.5

3. DATA CONSISTENCY CHECK:
   Total casualties mismatch: 0 records (0.00%)


,accident_date,day_of_week,accident_time,state_code,highway_number,kilometer_marker,city_name,accident_cause,accident_type,casualty_status,daylight_condition,traffic_direction,weather_condition,road_classification,road_geometry,total_people_involved,fatalities,minor_injuries,serious_injuries,uninjured_count,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station,accident_hour,calculated_total,total_match_ind
0,2017-01-01,sunday,01:45:00,RS,116.0,34.9,vacaria,Mechanical loss/defect of vehicle,rear-end collision,with injured victims,Night,Decreasing,clear sky,Simple,straight,6,0,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS,1,4,True
1,2017-01-01,sunday,01:00:00,PR,376.0,636.0,tijucas do sul,Incompatible velocity,run-off-road,with dead victims,Night,Increasing,drizzle,Double,curve,2,1,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR,1,0,True
2,2017-01-01,sunday,04:40:00,BA,101.0,65.0,entre rios,Driver was sleeping,head-on collision,with dead victims,Sunrise,Decreasing,cloudy,Simple,curve,5,1,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA,4,2,True
3,2017-01-01,sunday,06:30:00,PA,316.0,72.5,castanhal,Driver's lack of attention to conveyance,side impact collision,with dead victims,Sunrise,Decreasing,clear sky,Simple,straight,4,1,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA,6,0,True
4,2017-01-01,sunday,09:00:00,GO,20.0,220.5,posse,Road's defect,collision with fixed object,with injured victims,Day,Decreasing,clear sky,Simple,temporary detour,3,0,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF,9,3,True


In [ ]:
def create_engineered_features(df):
    """Create new features for enhanced analysis"""
    
    # Extract date components
    # YOUR CODE HERE: Extract year, month, and quarter from 'accident_date'
    
    # Create accident severity score
    # YOUR CODE HERE: Define a scoring system based on accident severity call it 'accident_severity_score'
    
    # Time of day categories function created for you to use
    def categorize_time(hour):
        if pd.isna(hour):
            return 'unknown'
        elif 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'afternoon'
        elif 18 <= hour < 22:
            return 'evening'
        else:
            return 'night'
    
    df['time_of_day_category'] = # YOUR CODE HERE: Apply categorize_time function to 'accident_time' 
    
    # Weekend flag
    df['weekend_flag'] = # YOUR CODE HERE: Check if 'day_of_week' is Saturday or Sunday
    
    # Fatal accident flag
    df['fatal_accident_flag'] = # YOUR CODE HERE: Check if 'fatalities' > 0
    
    # Multi-vehicle flag
    df['multi_vehicle_flag'] = # YOUR CODE HERE: Check if 'vehicle_count' > 1
    
    print("✓ Feature engineering completed")
    print(f"   Created {len(['year', 'month', 'quarter', 'accident_severity_score', 'time_of_day_category', 'weekend_flag', 'fatal_accident_flag', 'multi_vehicle_flag'])} new features")
    
    return df

In [104]:
df_featured = create_engineered_features(df_clean)
df_featured.head()

✓ Feature engineering completed
   Created 8 new features


,accident_date,day_of_week,accident_time,state_code,highway_number,kilometer_marker,city_name,accident_cause,accident_type,casualty_status,daylight_condition,traffic_direction,weather_condition,road_classification,road_geometry,total_people_involved,fatalities,minor_injuries,serious_injuries,uninjured_count,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station,accident_hour,calculated_total,total_match_ind,year,month,day,quarter,accident_severity_score,time_of_day_category,weekend_flag,fatal_accident_flag,multi_vehicle_flag
0,2017-01-01,sunday,01:45:00,RS,116.0,34.9,vacaria,Mechanical loss/defect of vehicle,rear-end collision,with injured victims,Night,Decreasing,clear sky,Simple,straight,6,0,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS,1,4,True,2017,1,1,1,4,night,True,False,True
1,2017-01-01,sunday,01:00:00,PR,376.0,636.0,tijucas do sul,Incompatible velocity,run-off-road,with dead victims,Night,Increasing,drizzle,Double,curve,2,1,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR,1,0,True,2017,1,1,1,10,night,True,True,True
2,2017-01-01,sunday,04:40:00,BA,101.0,65.0,entre rios,Driver was sleeping,head-on collision,with dead victims,Sunrise,Decreasing,cloudy,Simple,curve,5,1,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA,4,2,True,2017,1,1,1,14,night,True,True,True
3,2017-01-01,sunday,06:30:00,PA,316.0,72.5,castanhal,Driver's lack of attention to conveyance,side impact collision,with dead victims,Sunrise,Decreasing,clear sky,Simple,straight,4,1,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA,6,0,True,2017,1,1,1,10,morning,True,True,True
4,2017-01-01,sunday,09:00:00,GO,20.0,220.5,posse,Road's defect,collision with fixed object,with injured victims,Day,Decreasing,clear sky,Simple,temporary detour,3,0,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF,9,3,True,2017,1,1,1,5,morning,True,False,False


In [105]:
df_featured.columns

Index(['accident_date', 'day_of_week', 'accident_time', 'state_code',
       'highway_number', 'kilometer_marker', 'city_name', 'accident_cause',
       'accident_type', 'casualty_status', 'daylight_condition',
       'traffic_direction', 'weather_condition', 'road_classification',
       'road_geometry', 'total_people_involved', 'fatalities',
       'minor_injuries', 'serious_injuries', 'uninjured_count',
       'unknown_status', 'total_casualties', 'vehicle_count',
       'accident_latitude', 'accident_longitude', 'regional_office',
       'reporting_station', 'accident_hour', 'calculated_total',
       'total_match_ind', 'year', 'month', 'day', 'quarter',
       'accident_severity_score', 'time_of_day_category', 'weekend_flag',
       'fatal_accident_flag', 'multi_vehicle_flag'],
      dtype='object')

In [106]:
df_featured.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460929 entries, 0 to 463151
Data columns (total 39 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   accident_date            460929 non-null  datetime64[ns]
 1   day_of_week              460929 non-null  object        
 2   accident_time            460929 non-null  object        
 3   state_code               460929 non-null  object        
 4   highway_number           460929 non-null  float64       
 5   kilometer_marker         460929 non-null  float64       
 6   city_name                460929 non-null  object        
 7   accident_cause           460929 non-null  object        
 8   accident_type            460929 non-null  object        
 9   casualty_status          460929 non-null  object        
 10  daylight_condition       460929 non-null  object        
 11  traffic_direction        460929 non-null  object        
 12  weather_condition    

# Create Four Targeted Analysis DataFrames

## Analysis 1: Severity Analysis - What Conditions Lead to the Most Severe Accidents? 🚨
This analysis uses the powerful `accident_severity_score` to find out which conditions are associated with more dangerous accidents, not just more frequent ones.
* __Purpose__: Identify which weather and road combinations produce the most dangerous accidents (not just frequent ones)
* __Analysis__: Group by `weather_condition` and `road_classification`. Calculate the average `accident_severity_score` for each combination.
* __Resulting DataFrame Columns__: `weather_condition`, `road_classification`, `average_severity_score`

In [ ]:
def create_severity_analysis(df):
    """
    Analysis 1: Severity Analysis
    Find which conditions lead to the most severe accidents using accident_severity_score
    """
    
    # Group by weather and road conditions to calculate average severity score and count accidents
    severity_df = # YOUR CODE HERE
    
    # rename columns for clarity
    severity_df.columns = ['average_severity_score', 'accident_count']
    
    # Reset index to make grouping columns regular columns
    
    
    # Sort by severity score to identify most dangerous combinations
    
    
    # Final DataFrame with required columns
    severity_analysis = # YOUR CODE HERE: Create a DataFrame with 'weather_condition', 'road_classification', and 'average_severity_score'
    
    print("✓ Severity Analysis completed")
    print(f"   Found {len(severity_analysis)} weather/road combinations")
    print(f"   Most severe combination: {severity_analysis.iloc[0]['weather_condition']} + {severity_analysis.iloc[0]['road_classification']}")
    
    return severity_analysis

In [108]:
severity_analysis = create_severity_analysis(df_featured)
severity_analysis.head()

✓ Severity Analysis completed
   Found 28 weather/road combinations
   Most severe combination: fog + Simple


,weather_condition,road_classification,average_severity_score
11,fog,Simple,3.51
16,ignored,Simple,3.27
27,windy,Simple,3.04
19,rainy,Simple,2.98
2,clear sky,Simple,2.93


## Analysis 2: Temporal Patterns of Fatal Accidents - When are Fatal Accidents Most Common? 🌃
This analysis focuses specifically on the most tragic outcomes using the new flags.
* __Purpose__: Focus specifically on fatal accidents to understand when the most tragic outcomes occur
* __Analysis__: Filter the data for `fatal_accident_flag` == True. Then, group by `time_of_day_category` and `weekend_flag` and count the number of fatal accidents.
* __Resulting DataFrame Columns__: `time_of_day_category`, `weekend_flag`, `number_of_fatal_accidents`

In [ ]:
def create_fatal_temporal_analysis(df):
    """
    Analysis 2: Temporal Patterns of Fatal Accidents
    Filter for fatal accidents and analyze timing patterns
    """
    
    # Filter for fatal accidents only
    # YOUR CODE HERE: Filter DataFrame for fatal accidents where 'fatal_accident_flag' is True
    fatal_accidents = # YOUR CODE
    # YOUR CODE HERE: Check if there are any fatal accidents, if there are no fatal accident print "⚠️ No fatal accidents found in dataset"
     
    # Group by time and weekend patterns (time_of_day_category and weekend_flag) and capture the count of fatal accidents
    fatal_temporal_df = fatal_accidents.groupby # YOUR CODE HERE. 

    # rename columns for clarity
    fatal_temporal_df.columns = ['time_of_day_category', 'weekend_flag', 'number_of_fatal_accidents']
    
    # Sort by number of fatal accidents for insights
    fatal_temporal_df = # YOUR CODE HERE: Sort the DataFrame by 'number_of_fatal_accidents' in descending order
    
    print("✓ Fatal Temporal Analysis completed")
    print(f"   Total fatal accidents analyzed: {len(fatal_accidents)}")
    print(f"   Peak fatal accident period: {fatal_temporal_df.iloc[0]['time_of_day_category']} ({'weekend' if fatal_temporal_df.iloc[0]['weekend_flag'] else 'weekday'})")
    
    return fatal_temporal_df

In [110]:
fatal_temporal_analysis = create_fatal_temporal_analysis(df_featured)
fatal_temporal_analysis.head()

✓ Fatal Temporal Analysis completed
   Total fatal accidents analyzed: 31212
   Peak fatal accident period: evening (weekday)


,time_of_day_category,weekend_flag,number_of_fatal_accidents
2,evening,False,5421
6,night,False,4874
0,afternoon,False,4676
7,night,True,4014
4,morning,False,4012


## Analysis 3: Accident Cause and Vehicle Involvement 🚗
This analysis explores the relationship between the primary cause of an accident and whether it involved single or multiple vehicles.
* __Purpose__: Explore relationship between accident causes and single vs multi-vehicle involvement
* __Analysis__: Group by accident_cause and `multi_vehicle_flag`. Count the number of accidents and also calculate the average accident_severity_score.
* __Resulting DataFrame Columns__: `accident_cause`, `multi_vehicle_flag`, `number_of_accidents`, `average_severity_score`


In [ ]:
def create_cause_vehicle_analysis(df):
    """
    Analysis 3: Accident Cause and Vehicle Involvement
    Analyze relationship between causes and multi-vehicle involvement
    """
    
    # Group by accident cause and vehicle involvement (accident_cause and multi_vehicle_flag) and calculate average severity score and count accidents
    
    # rename columns for clarity
    cause_vehicle_df.columns = ['number_of_accidents', 'average_severity_score']
    
    # Reset index to make grouping columns regular columns
    cause_vehicle_df = # YOUR CODE
    
    # Sort by number of accidents for primary insights
    cause_vehicle_df = # YOUR CODE
    
    print("✓ Cause-Vehicle Analysis completed")
    print(f"   Analyzed {len(cause_vehicle_df)} cause/vehicle combinations")
    
    # Show most common cause for single vs multi-vehicle
    single_vehicle = # YOUR CODE
    multi_vehicle = # YOUR CODE
    
    if len(single_vehicle) > 0:
        print(f"   Most common single-vehicle cause: {single_vehicle.iloc[0]['accident_cause']}")
    if len(multi_vehicle) > 0:
        print(f"   Most common multi-vehicle cause: {multi_vehicle.iloc[0]['accident_cause']}")
    
    return cause_vehicle_df

In [115]:
cause_vehicle_analysis = create_cause_vehicle_analysis(df_featured)
cause_vehicle_analysis.head()

✓ Cause-Vehicle Analysis completed
   Analyzed 169 cause/vehicle combinations
   Most common single-vehicle cause: Driver's lack of attention to conveyance
   Most common multi-vehicle cause: Driver's lack of attention to conveyance


,accident_cause,multi_vehicle_flag,number_of_accidents,average_severity_score
53,Driver's lack of attention to conveyance,True,70972,2.25
52,Driver's lack of attention to conveyance,False,36583,1.79
74,Incompatible velocity,False,31471,2.19
35,Driver broke the laws of transit,True,25790,2.99
54,Driver's lack of reaction,False,20662,2.05


## Analysis 4: Quarterly Accident Hotspot Analysis 📍
This analysis uses the location and time data to find seasonal accident hotspots.
* __Purpose__: Identify seasonal accident patterns by state to find regional hotspots
* __Analysis__: Group by `state_code` and quarter. Count the number of accidents to identify if certain states have accident seasons.
* __Resulting DataFrame Columns__: `state_code`, `quarter`, `total_accidents`

In [ ]:
def create_quarterly_hotspot_analysis(df):
    """
    Analysis 4: Quarterly Accident Hotspot Analysis
    Find seasonal accident hotspots by state
    """
    
    # group by state_code and quarter to count accidents
    # reset index to make grouping columns regular columns
    # YOUR CODE
    
    # rename columns for clarity
    hotspot_df.columns = ['state_code', 'quarter', 'total_accidents']
    
    # Sort by total accidents to identify hotspots
    hotspot_df = # YOUR CODE
    
    # Add quarter labels for better interpretation
    quarter_labels = # create a mapping for quarter numbers to names for example {1: 'Q1 (Jan-Mar)'}
    hotspot_df['quarter_name'] = hotspot_df['quarter'].map(quarter_labels)
    
    print("✓ Quarterly Hotspot Analysis completed")
    print(f"   Analyzed {len(df['state_code'].unique())} states across 4 quarters")
    
    # Identify top hotspot
    top_hotspot = # YOUR CODE: Get the row with the highest total accidents
    print(f"   Top seasonal hotspot: {top_hotspot['state_code']} in {top_hotspot['quarter_name']} ({top_hotspot['total_accidents']} accidents)")
    
    # Final DataFrame with required columns (removing quarter_name as it's just for logging)
    final_hotspot_df = hotspot_df[['state_code', 'quarter', 'total_accidents']].copy()
    
    return final_hotspot_df

In [117]:
quarterly_hotspot_analysis = create_quarterly_hotspot_analysis(df_featured)
quarterly_hotspot_analysis.head()

✓ Quarterly Hotspot Analysis completed
   Analyzed 27 states across 4 quarters
   Top seasonal hotspot: MG in Q1 (Jan-Mar) (16230 accidents)


,state_code,quarter,total_accidents
40,MG,1,16230
42,MG,3,15146
43,MG,4,14961
41,MG,2,14829
92,SC,1,14463


## Complete Data Processing Pipeline (putting it all together)

In [ ]:
def main_analysis_pipeline():
    """
    Complete pipeline from S3 to Snowflake with four targeted analyses
    """
    
    print("🚀 Starting UK Traffic Accident Analysis Pipeline")
    print("=" * 60)
    
    # Step 1: Load data
    df = # YOUR CODE
    
    # Step 2: Rename columns
    df =  # YOUR CODE
    
    # Step 3: Fix data types
    df =  # YOUR CODE
    
    # Step 4: Data quality checks
    df_clean =  # YOUR CODE
    
    # Step 5: Feature engineering
    df_final =  # YOUR CODE
    
    print("\n🔍 Creating Four Targeted Analyses")
    print("=" * 40)
    
    # Step 6: Create four analysis DataFrames using the df_final DataFrame
    analysis_1 =  # YOUR CODE
    analysis_2 = # YOUR CODE
    analysis_3 = # YOUR CODE
    analysis_4 = # YOUR CODE
    
    return analysis_1, analysis_2, analysis_3, analysis_4, df_final

# Run the complete pipeline
severity_df, fatal_temporal_df, cause_vehicle_df, hotspot_df, cleaned_data = main_analysis_pipeline()

🚀 Starting UK Traffic Accident Analysis Pipeline
Loaded 463152 records with 27 columns
✓ Columns renamed successfully
✓ Data types fixed and standardized
=== DATA QUALITY REPORT ===

1. MISSING VALUES:
Columns with missing values:
   reporting_station: 1310 missing (0.28%)
   highway_number: 990 missing (0.21%)
   kilometer_marker: 990 missing (0.21%)
   regional_office: 10 missing (0.00%)

   Removed 2223 rows with missing data

2. OUTLIER ANALYSIS:
   total_people_involved: 8545 outliers (1.85%)
      Max value: 80, Expected max: 6.0
   fatalities: 31212 outliers (6.77%)
      Max value: 21, Expected max: 0.0
   minor_injuries: 20205 outliers (4.38%)
      Max value: 61, Expected max: 2.5
   serious_injuries: 96315 outliers (20.90%)
      Max value: 31, Expected max: 0.0
   total_casualties: 30354 outliers (6.59%)
      Max value: 66, Expected max: 2.5
   vehicle_count: 8558 outliers (1.86%)
      Max value: 23, Expected max: 3.5

3. DATA CONSISTENCY CHECK:
   Total casualties mismat

# Snowflake Integration
* Push all four datasets into Snowflake and save them as tables (transient or standard)

## Four Snowflake Temporary Tables:

* __SEVERITY_ANALYSIS__:
    * Columns: weather_condition, road_classification, average_severity_score
    * Purpose: Risk assessment for infrastructure and safety planning


* __FATAL_TEMPORAL_PATTERNS__
    * Columns: time_of_day_category, weekend_flag, number_of_fatal_accidents
    * Purpose: Emergency service resource allocation and enforcement timing


* __CAUSE_VEHICLE_ANALYSIS__:
    * Columns: accident_cause, multi_vehicle_flag, number_of_accidents, average_severity_score
    * Purpose: Prevention strategy development and driver education programs

* __QUARTERLY_HOTSPOT_ANALYSIS__:
    * Columns: state_code, quarter, total_accidents
    * Purpose: Seasonal safety campaign planning and resource allocation

In [ ]:
# Your code to save the DataFrames to Snowflake or any other destination
# use a snow.cfg file to configure the connection
# call the section [DEV]

import pandas as pd
from snowflake import connector
from configparser import ConfigParser

config = ConfigParser()
config.read('snow.cfg')

config.sections()

In [ ]:
params = dict(config['DEV'])

In [ ]:
# After you write your data, write SQL queries to verify the data
# For example, you can use the following SQL query to check the number of records in each 
# SELECT COUNT(*) FROM <your_table_name>; for each of the DataFrames you created